In [ ]:
import json, os
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd

test_name = 'neuroM_morph_softChecks'
model_results = 'population_models_excerpt-pop/20191219-163146'
val_results_dir='./validation_results'
val_results_path = os.path.join(val_results_dir, test_name, model_results)

if not os.path.exists(val_results_path):
    os.makedirs(val_results_path)
    
pred_file_path = os.path.join(val_results_path,'prediction_summary_FSI_pop.json')
with open(pred_file_path, 'r') as fp:
    pop_prediction_raw = json.load(fp)

In [ ]:
prediction_raw = pop_prediction_raw.values()[0]

pred_CellPart_df = dict()
pred_CellPart_df_list = list()
for CellPart in prediction_raw.keys():
    pred_CellPart_df[CellPart] = pd.DataFrame(prediction_raw[CellPart])

In [ ]:
def feats_to_plot(all_feats=[], feats_exc=[]):
    for feat_name in feats_exc:
        try:
            all_feats.remove(feat_name)
        except:
            pass

    return all_feats

In [ ]:
def corrfunc(x, y, **kws):
    r,p = stats.pearsonr(x,y)
    ax = plt.gca()
    ax.annotate("r = {:.2E}\n(p ={:.2E})".format(r,p),
                xy=(.1, .9), xycoords=ax.transAxes)

In [ ]:
feats_exc = ['total_number_of_neurites']
for CellPart in prediction_raw.keys():
    prediction_raw_df=pred_CellPart_df[CellPart]
    all_feats = feats_to_plot(all_feats=list(prediction_raw_df), feats_exc=feats_exc)
        
    print '{} -> {} \n'.format(CellPart, all_feats)    
    data = prediction_raw_df.loc[:, all_feats]    
    g = sns.pairplot(data, height=5, aspect=1, diag_kind="kde")
    # g = sns.PairGrid(data, size=5, aspect=1, palette=["red"])
    # g.map(sns.regplot)
    # g.map(corrfunc)
    g.map_upper(sns.regplot)
    g.map_upper(corrfunc)
    g.map_diag(sns.distplot, kde=True)
    g.map_lower(sns.regplot)
    g.map_lower(corrfunc)
    
    plt.subplots_adjust(top=0.95)
    g.fig.suptitle('Cell part: '+CellPart, fontsize=17) 

    filepath = os.path.join(val_results_path,'prediction_lreg_'+CellPart+'_FSI_pop.pdf')
    # plt.savefig(filepath, dpi=600, )

In [ ]:
feats_exc = ['total_number_of_neurites', 'max_section_branch_order', 'total_soma_radii']
for CellPart in prediction_raw.keys():
    
    all_feats = feats_to_plot(all_feats=list(prediction_raw_df), feats_exc=feats_exc)
    
    print '{} -> {} \n'.format(CellPart, all_feats)    

    data = prediction_raw_df.loc[:, all_feats]
    g = sns.pairplot(data, height=5, aspect=1, diag_kind="kde")
    # g = sns.PairGrid(data, size=5, aspect=1, palette=["red"])
    g.map_upper(sns.regplot)
    g.map_upper(corrfunc)
    # g.map_upper(plt.reg, s=10)
    g.map_diag(sns.distplot, kde=True)
    g.map_lower(sns.kdeplot, cmap="Blues_d", n_levels=8)
    
    plt.subplots_adjust(top=0.95)
    g.fig.suptitle('Cell part: ' + CellPart, fontsize=17) 
    
    filepath = os.path.join(val_results_path,'prediction_stats__'+CellPart+'_FSI_pop.pdf')
    # plt.savefig(filepath, dpi=600, )

In [ ]:
pred_CellPart_df['neuron'].head()
pred_CellPart_df['neuron'][['max_soma_radii', 'total_soma_radii']]
pred_CellPart_df['neuron'].corr().abs()

In [ ]:
"""    
with sns.axes_style('white'):
    for x,y in zip(x_vars,y_vars):
        sns.jointplot(x, y, data.loc[data.NeuriteType=='axon'], kind='reg')
"""

In [ ]:
pred_CellPart_df['basal_dendrite'][['total_number_of_neurites']]